In [ ]:
%cd ../..
%load_ext autoreload

%autoreload 2

In [ ]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from rts.features.pose import load_all_poses, KEYPOINTS_NAMES
from rts.utils import dataframe_from_hdf5

poses_folder = "data/test_poses/"
archiva_path = "/mnt/g/ioc/data/"

In [ ]:
poses = load_all_poses(poses_folder)

In [ ]:
data = dataframe_from_hdf5(archiva_path, "metadata")

poses_df = pd.DataFrame(poses)
poses_df = pd.merge(poses_df, data[["seq_id", "sport"]], left_on="video_name", right_on="seq_id")
poses_df.head()

# Missing keypoints

In [ ]:
poses_df["n_missing_keypoints"] = poses_df.keypoints.map(lambda x: len([k for k in x if k[2] == 0]))
poses_df["n_missing_keypoints"].value_counts().sort_index().plot(kind="bar")
plt.xticks(rotation=0)  
plt.title("Number of poses with at least N missing keypoints")
plt.show()

In [ ]:
top_sports = poses_df.sport.value_counts().head(20).index
plt.figure(figsize=(20,10))
sns.boxplot(x="sport", y="n_missing_keypoints", data=poses_df[poses_df.sport.isin(top_sports)])
plt.xticks(rotation=45)
plt.show()

In [ ]:
poses_df["n_miss_upper"] = poses_df.keypoints.map(lambda x: len([k for k in x[:13] if k[2] == 0])) # From top to hips included
poses_df["n_miss_lower"] = poses_df.keypoints.map(lambda x: len([k for k in x[13:] if k[2] == 0])) # Below hips (not included)

fig, ax = plt.subplots(1,2, figsize=(16,6))
poses_df["n_miss_upper"].value_counts().sort_index().plot(kind="bar", ax=ax[0])
ax[0].set_title("Missing keypoints (Upper body)")
ax[0].set_xlabel("Number of missing keypoints")
ax[0].set_ylabel("Count")
ax[0].set_xticklabels(ax[0].get_xticklabels(), rotation=0)

poses_df["n_miss_lower"].value_counts().sort_index().plot(kind="bar", ax=ax[1])
ax[1].set_title("Missing keypoints (Lower body)")
ax[1].set_xlabel("Number of missing keypoints")
ax[1].set_ylabel("Count")
ax[1].set_xticklabels(ax[1].get_xticklabels(), rotation=0)

plt.show()


In [ ]:
# For each keypoint, compute the number of missing keypoints
keypoints_df_scores = pd.DataFrame(poses_df["keypoints"].apply(lambda x: [k[2] for k in x]).tolist(), columns=KEYPOINTS_NAMES)
plt.figure(figsize=(12,10))
keypoints_df_scores.boxplot(column=KEYPOINTS_NAMES, vert=False)
plt.title("Boxplot of confidence score for each keypoint")
plt.show()


In [ ]:
# For each keypoint, compute the number of missing keypoints
keypoints_df_missing = pd.DataFrame(keypoints_df_scores.apply(lambda x: x == 0), columns=KEYPOINTS_NAMES)
(keypoints_df_missing.sum() / len(keypoints_df_missing)).plot(kind="barh")
plt.title("Percentage of missing keypoints")
plt.xlim(0,1)
plt.show()